In [33]:
from langchain.chains import RetrievalQA
from langchain_community.llms import CTransformers
from langchain.memory import ConversationBufferWindowMemory

# Setup manually (similar to app.py)
memory = ConversationBufferWindowMemory(k=3)
llm = CTransformers(model=".\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512, 'temperature': 0.9})
chain_type_kwargs={"prompt": PROMPT, "memory": ConversationBufferWindowMemory(memory_key="history", input_key="question")}
question_answer_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs=chain_type_kwargs,
    memory=memory,
    verbose=True
)

# Test
result = question_answer_chain.invoke({"query": "What is a business model?"})
print(result)


NameError: name 'PROMPT' is not defined

In [9]:
loader=PyPDFDirectoryLoader("data")
data=loader.load()

In [10]:
data

[Document(page_content=" \n  Page | 1 \n \nE-commerce - Business models \nEvery business operates on business model of its own. Business model defines how a company \nruns its operations & generates revenue. Every viable organization is built on a sound business \nmodel. Selecting an ecommerce business model is a challenge, especially for beginners who \nhave little to no experience in the industry. If a business model is successfully executed, an \necommerce venture can become a significant source of income. Let us go through some most \nprevalent e-commerce business models in detail: \n1. B2B Business model \nA website following the B2B business model sells its products to an intermediate buyer who \nthen sells the product to the final customer. As an example, a wholesaler places an order from a \ncompany's website and after receiving the consignment, sells the end product to the final \ncustomer who comes to buy the product at one of its retail outlets.  \n \n \n  \n \n \n \nExample

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [12]:
len(text_chunks)

25

In [13]:
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("OPENAI_API_KEY")

In [17]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
from pinecone import Pinecone

In [19]:
pinecone_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_key)

In [21]:
index = pc.Index("chatbot-2")

In [23]:
index_name = "chatbot-2"
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real")
   index.describe_index_stats()

In [25]:
from langchain.vectorstores import Pinecone

text_field = 'text'
vectorstore = Pinecone(
    index, embeddings, text_field
)

c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


In [26]:
from sentence_transformers import SentenceTransformer
import pinecone

# Initialize the Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Your query
query = "What is a business model?"

# Generate the embedding using the SentenceTransformer model
xq = model.encode(query).tolist()

# Assuming you have already initialized the Pinecone index
# and created a `pinecone.Index` instance named `index`

dres = index.query(vector=xq, top_k=2, namespace='real', include_metadata=True)

# Print the results
print(dres)


c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'matches': [{'id': '0',
              'metadata': {'text': 'Page | 1 \n'
                                   ' \n'
                                   'E-commerce - Business models \n'
                                   'Every business operates on business model '
                                   'of its own. Business model defines how a '
                                   'company \n'
                                   'runs its operations & generates revenue. '
                                   'Every viable organization is built on a '
                                   'sound business \n'
                                   'model. Selecting an ecommerce business '
                                   'model is a challenge, especially for '
                                   'beginners who \n'
                                   'have little to no experience in the '
                                   'industry. If a business model is '
                                   'successful

In [27]:
pip install sentence-transformers transformers

Note: you may need to restart the kernel to use updated packages.


In [28]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pinecone
import time

# Load the embedding model
#embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load the local LLaMA-2 model
llama_model_path = "model/llama-2-7b-chat.ggmlv3.q4_0.bin"
tokenizer = AutoTokenizer.from_pretrained(llama_model_path)
model = AutoModelForCausalLM.from_pretrained(llama_model_path)

# Initialize text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)

limit = 3750

def retrieve(query):
    # Generate embeddings using the open-source model
    xq = embedding_model.encode([query])[0]

    # Retrieve from Pinecone
    contexts = []
    time_waited = 0
    while (len(contexts) < 3 and time_waited < 60 * 12):
        res = index.query(vector=xq.tolist(), top_k=3, namespace='real', include_metadata=True)
        contexts = contexts + [
            x['metadata']['text'] for x in res['matches']
        ]
        print(f"Retrieved {len(contexts)} contexts, sleeping for 15 seconds...")
        time.sleep(15)
        time_waited += 15

    if time_waited >= 60 * 12:
        print("Timed out waiting for contexts to be retrieved.")
        contexts = ["No contexts retrieved. Try to answer the question yourself!"]

    # Build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n" +
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # Append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts) - 1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

def complete(prompt):
    # Generate completion using the local LLaMA-2 model
    completion = generator(prompt, max_length=512, num_return_sequences=1)
    return completion[0]['generated_text'].strip()

# Example usage
query = "What is a Transformer?"
prompt = retrieve(query)
answer = complete(prompt)
print(answer)


OSError: model/llama-2-7b-chat.ggmlv3.q4_0.bin is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [30]:
pip install llama-cpp-python



     ---------------------------------------- 0.0/63.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/63.7 MB ? eta -:--:--
     --------------------------------------- 0.0/63.7 MB 393.8 kB/s eta 0:02:42
     --------------------------------------- 0.1/63.7 MB 819.2 kB/s eta 0:01:18
     ---------------------------------------- 0.2/63.7 MB 1.4 MB/s eta 0:00:47
     ---------------------------------------- 0.5/63.7 MB 2.3 MB/s eta 0:00:28
      --------------------------------------- 0.9/63.7 MB 3.7 MB/s eta 0:00:17
     - -------------------------------------- 2.0/63.7 MB 6.6 MB/s eta 0:00:10
     -- ------------------------------------- 3.2/63.7 MB 9.2 MB/s eta 0:00:07
     -- ------------------------------------- 4.3/63.7 MB 10.9 MB/s eta 0:00:06
     --- ------------------------------------ 5.5/63.7 MB 12.6 MB/s eta 0:00:05
     ---- ----------------------------------- 6.7/63.7 MB 13.7 MB/s eta 0:00:05
     ----- ---------------------------------- 8.1/63.7 MB 15

In [31]:
from sentence_transformers import SentenceTransformer
import pinecone

# Initialize the Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Your query
query = "What is a Transformer?"

# Generate the embedding using the SentenceTransformer model
xq = model.encode(query).tolist()

# Assuming you have already initialized the Pinecone index
# and created a `pinecone.Index` instance named `index`

dres = index.query(vector=xq, top_k=2, namespace='real', include_metadata=True)

# Print the results
print(dres)


{'matches': [{'id': '13',
              'metadata': {'text': 'Page | 4 \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   'jdj \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   '4. C2B business model \n'
                                   'In this model, a consumer approaches a '
                                   'website showing multiple business '
                                   'organizations for a

In [32]:
query = (
    "What is business " +
    "in the business model"
)

query_with_contexts = retrieve(query)
query_with_contexts

NameError: name 'retrieve' is not defined